In [2]:
# Install necessary libraries
!pip install transformers
!pip install torch
!pip install ipywidgets
!pip install newspaper3k


In [4]:
!pip install "lxml[html_clean]"
# Import necessary libraries
from transformers import BartForConditionalGeneration, BartTokenizer
import ipywidgets as widgets
from IPython.display import display
from newspaper import Article

In [5]:

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [6]:

# Define the summarize function
def summarize(text, model, tokenizer, max_length=150, min_length=40, length_penalty=2.0, num_beams=4):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [7]:

# Define the fetch article function
def fetch_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

In [8]:

# Create input and output widgets
input_text = widgets.Textarea(
    value='',
    placeholder='Enter news article text here...',
    description='News Article:',
    layout=widgets.Layout(width='100%', height='200px')
)

input_url = widgets.Text(
    value='',
    placeholder='Enter URL here...',
    description='Article URL:',
    layout=widgets.Layout(width='100%')
)

output_text = widgets.Textarea(
    value='',
    placeholder='Summary will appear here...',
    description='Summary:',
    layout=widgets.Layout(width='100%', height='200px')
)

button = widgets.Button(description='Summarize')

In [9]:

# Define the button click event
def on_button_click(b):
    output_text.value = 'Summarizing...'
    if input_url.value:
        article_text = fetch_article(input_url.value)
    else:
        article_text = input_text.value
    summary = summarize(article_text, model, tokenizer)
    output_text.value = summary

button.on_click(on_button_click)

In [10]:

# Display the widgets
display(input_url, input_text, button, output_text)


Text(value='', description='Article URL:', layout=Layout(width='100%'), placeholder='Enter URL here...')

Textarea(value='', description='News Article:', layout=Layout(height='200px', width='100%'), placeholder='Ente…

Button(description='Summarize', style=ButtonStyle())

Textarea(value='', description='Summary:', layout=Layout(height='200px', width='100%'), placeholder='Summary w…